In [1]:
# todo
# -> dynamic colum name generation depending on the number of #N

In [ ]:
import re
import pdftotext
import pandas as pd

In [2]:
#path_in = '/home/philipp/Data/obf_autman/pdf/Manuale_Johannklause_20201109.pdf'
path_in = '/home/philipp/Data/obf_autman/pdf/Manuale_Dross_Abt.156_182.pdf'
#path_out = '/home/philipp/Data/obf_autman/csv/Manuale_Dross_Abt.156_182.csv'
path_out = '/home/philipp/Data/obf_autman/csv/Manuale_Dross_Abt.156_182.xlsx'
max_pages = 0

In [3]:
def other_tool(filename, end):
    data = []
    with open(filename, "rb") as f:
        pdf = pdftotext.PDF(f)  
    for i, page in enumerate(pdf) :
        data.append({'page': i , 'content': page})
        if i == end:
            break
    return data

In [4]:
#path_in = '/home/philipp/Data/obf_autman/pdf/20201203_PD_Manuale_Pernitz_test.pdf'

In [5]:
with open(path_in, "rb") as f:
    pdf = pdftotext.PDF(f)
    max_pages = len(pdf)
    print(max_pages)

532


In [6]:
pdf_data = other_tool(path_in, max_pages)

In [7]:
pdf_data

[{'page': 0,
  'content': ' FB 172\n FR 02\n          Forstbetrieb Waldviertel Voralpen\n          Droß\n                                                         Druckdatum: 20.06.2021               BKL       Umtrieb        Wirtschaftswald\n                                                                                                                                                      156 A 0\n TO: 1420 Laufzeit: 01.01.2022- 31.12.2031\n                                                          Seite 1 von 523                    9010         100          außer Ertrag\n                                                                                                                                                                 1,47 ha\n    Seeh [m]         Exp        Neig [%]         Neig [°]            STOE              VTYP          Wald-Typ       Wuchs-Geb. Verbissgrad           Schälgrad       fr.SS\n      300            W            36                20                 61        

In [23]:
txt = pdf_data[101]['content']

In [8]:
'''for typ in ['#BZ', '#T', '#N', '#G']:
    start = txt.find(typ)
    print(start)
    end = txt[start+1:].find('#') + start + 1
    print(txt[start:end])'''

"for typ in ['#BZ', '#T', '#N', '#G']:\n    start = txt.find(typ)\n    print(start)\n    end = txt[start+1:].find('#') + start + 1\n    print(txt[start:end])"

In [9]:
txt

' FB 172\n FR 02\n          Forstbetrieb Waldviertel Voralpen\n          Droß\n                                                                       Druckdatum: 20.06.2021                                    BKL              Umtrieb                Wirtschaftswald\n                                                                                                                                                                                                          257 R 1\n TO: 1420 Laufzeit: 01.01.2022- 31.12.2031\n                                                                        Seite 101 von 170                                      9010                  120                 in Ertrag\n                                                                                                                                                                                                                     0,64 ha\n    Seeh [m]         Exp            Neig [%]                Neig [°]          

In [10]:
x = get_wo(txt)
x

NameError: name 'get_wo' is not defined

In [59]:
x.find('Standortschutzwald\n')

-1

In [45]:
x[166:]

'055 A 2'

In [10]:
x = txt[txt.find('Wirtschaftswald\n')+20:txt.find('\n TO')].strip()

In [11]:
x

'053 M 4'

In [8]:
check_hashtype = ['#U', '#W', '#STOE', '#VTYP', '#VB', '#VG', '#UENH', '#UELH', '#BZ', '#PZ', '#ST',\
                '#BE', '#MA', '#SWE', '#LRV', '#WO', '#N']

check_u = [80, 100, 120, 140, 160, 200]
check_w = ['WI', 'WA', 'SSI', 'SSA', 'OSI', 'OSA', 'BSI', 'BSA']
check_stoe = [1, 2, 3, 4, 5, 6, 7, 8, 9, 11, 12, 13, 14, 15, 21, 22, 23, 25, 26, 27, 28, 31, 32, 33, 34, 35, \
              36, 37, 41, 42, 43, 44, 51, 52, 53, 54, 55, 56, 57, 58, 61, 62, 63, 64, 65, 66, 71, 72, 73, 74, \
              75, 76, 81, 82, 83, 84, 85, 86, 86, 87, 88, 89, 91, 92, 93, 94, 95, 96, 97, 98]
check_ma = ['AD', 'AE', 'AF', 'AG', 'AS', 'BU', 'DE', 'DF', 'DP', 'EG', 'FM', 'JF', 'JP', 'KE', 'KF', 'KH', \
            'LI', 'LL', 'NB', 'ND', 'PA', 'PL', 'RM', 'SB', 'TR', 'UE', 'ZE', 'ZN', 'ZV']
check_rueck = [0,10,23,26,29,30,31,35,36,38,40,90]

check_swe = ['grün', 'gelb', 'rot', 'Grün', 'Gelb', 'Rot']

## Get data

In [9]:
def check_note(note, typ):
    if note == '':
        return 'pass'
    
    elif typ == '#U ':
        try:
            if int(note) in check_u:
                return 'pass'
            else:
                return 'error: #U - Umtriebszeit not valid'
        except ValueError:
            return 'error: #U - Umtriebszeit not valid'
    
    elif typ == '#W ':
        if note in check_w:
            return 'pass'
        else:
            return 'error: #W - Ertragsituation not valid'
    
    elif typ == '#STOE ':
        try:
            if int(note) in check_stoe:
                return 'pass'
            else:
                return 'error: #STOE - STOE not valid'
        except ValueError:
            return 'error: #STOE - STOE not valid'
        
    elif (typ == '#VB ') or (typ == '#VG '):
        try:
            if (int(note) > 0) and (int(note) < 6):
                return 'pass'
            else:
                return 'error: #VB - VB not valid'
        except ValueError:
            return 'error: #VB - VB not valid'
        
    elif (typ == '#UENH ') or (typ == '#UELH '):
        try:
            if int(note) > 0:
                return 'pass'
            else:
                return 'error: #UENH/UELH - UE not valid'
        except ValueError:
            return 'error: #UENH/UELH - UE not valid'
        
    elif typ == '#SWE ':
        try:
            if note in check_swe:
                return 'pass'
            else:
                return 'error: #SWE - SWE not valid'
        except ValueError:
            return 'error: #SWE - SWE not valid'
        
    return 'pass'

In [10]:
def check_nutz(element, typ):
    
    if typ == 0:
        try:
            if int(element) < 10:
                return 'pass'
            else:
                return 'error: #N - Schicht not valid'
        except ValueError:
            return 'error: #N - Schicht not valid'
        
    elif typ == 1:
        if element in check_ma:
            return 'pass'
        else:
            return 'error: #N - Maßnahme not valid'
        
    elif typ == 2:
        return 'pass'
    # area -> replace , to . -> float
    elif typ == 3:
        try:
            if int(element) >= 0:
                return 'pass'
            else:
                return 'error: #N - LH-Nutzungsmasse not valid'
        except ValueError:
            return 'error: #N - LH-Nutzungsmasse not valid'
        
    elif typ == 4:
        try:
            if int(element) >= 0:
                return 'pass'
            else:
                return 'error: #N - NH-Nutzungsmasse not valid'
        except ValueError:
            return 'error: #N - NH-Nutzungsmasse not valid'
        
    elif typ == 5:
        try:
            if (int(element) > 0) & (int(element) < 7):
                return 'pass'
            else:
                return 'error: #N - Dringlichkeit not valid'
        except ValueError:
            return 'error: #N - Dringlichkeit not valid'
        
    elif typ == 6:
        try:
            if (int(element) > 0) & (int(element) < 4):
                return 'pass'
            else:
                return 'error: #N - Bewilligung not valid'
        except ValueError:
            return 'error: #N - Bewilligung not valid'
        
    elif typ == 7:
        try:
            if (int(element) > 0) & (int(element) < 4):
                return 'pass'
            else:
                return 'error: #N - Zeitpunkt not valid'
        except ValueError:
            return 'error: #N - Zeitpunkt not valid'
        
    elif typ == 8:
        try:
            if (int(element) >= 0) & (int(element) < 7):
                return 'pass'
            else:
                return 'error: #N - Schlägerung not valid'
        except ValueError:
            return 'error: #N - Schlägerung not valid'
        
    elif typ == 9:
        try:
            if int(element) in check_rueck:
                return 'pass'
            else:
                return 'error: #N - Rückung not valid'
        except ValueError:
            return 'error: #N - Rückung not valid'
        
    elif typ == 10:
        return 'pass'

In [11]:
# find #typ in text
def find_note(txt, typ, offset):
    '''
    input:
        method to find #typ in text
        txt (string) plain text
        typ (string) type of mark e.g. '#BZ'
        offset (int) starting point for search in text
    output:
        marked text (string)
        offset (integer) starting point for next iterration
    '''
    # find position (index) of #typ in text
    start = txt[offset:].find(typ)
    # if #typ found
    if start > -1:
        start += offset
        # find end of #typ
        if txt[start+1:].find('#') > 0:
            end = txt[start+1:].find('#') + start + 1
            # set offset to end -> starting point for next iterration
            offset = end
            return txt[start:end], 'pass', offset
        else:
            return '', 'error: {} - missing #'.format(typ), -1
    else:
        return '', 'pass', -1

In [12]:
# find #typ in text
def find_nutz(txt = '1 LI 1,5 0 160 1 2 2 4 35 Am O Rand'):
    offset = 0
    nutz = []
    check_n = 'pass'
    for i in range(10):
        end = txt[offset:].find(' ')
        if end != -1:
            element = txt[offset:offset+end]
            check_temp = check_nutz(element, i)
            if check_temp != 'pass':
                check_n = check_temp
            nutz.append(element)
            offset += end+1
        else:
            nutz.append('')
    nutz.append(txt[offset:])
    return nutz, check_n

In [13]:
# find #typ in text
def find_nutz(txt = '1 LI 1,5 0 160 1 2 2 4 35 Am O Rand'):
    offset = 0
    nutz = []
    check_n = 'pass'
    nutz_text = True
    for i in range(10):
        end = txt[offset:].find(' ')
        if end != -1:
            element = txt[offset:offset+end]
            check_temp = check_nutz(element, i)
            if check_temp != 'pass':
                check_n = check_temp
            nutz.append(element)
            offset += end+1
        else:            
            # try if rest is an integer append as nutz-type
            try:
                nutz.append(int(txt[offset:]))
                nutz_text = False
            
            except:
                nutz_text = True
    # if rest is not an integer append as nutz-text
    if nutz_text:
        nutz.append(txt[offset:])
    else:
        nutz.append('')
    return nutz, check_n

In [14]:
def get_wo(txt):
    if txt.find('Wirtschaftswald\n') >= 0:
        wo = txt[txt.find('Wirtschaftswald\n')+17:txt.find('\n TO')].strip()
    elif txt.find('Standortschutzwald\n') >= 0:
        wo = txt[txt.find('Standortschutzwald\n')+166:txt.find('\n TO')].strip()
    elif txt.find('Objektschutzwald\n') >= 0:
        wo = txt[txt.find('Objektschutzwald\n')+164:txt.find('\n TO')].strip()
    elif txt.find('Bannwald\n') >= 0:
        wo = txt[txt.find('Bannwald\n')+156:txt.find('\n TO')].strip()
        
    return wo

In [15]:
def get_data_page(txt):
    '''
    input:
        search page for all #typ
        txt (string) plain text
    output:
        data (dataframe) containing the data
    '''
    data = []
    data.append('pass')
    # add WO
    data.append(get_wo(txt))
    # loop over all #typ
    for typ in ['#U ', '#W ', '#STOE ', '#VTYP ', '#VB ', '#VG ', '#UENH ', '#UELH ', \
                '#BZ ', '#PZ ', '#ST ', '#BE ', '#MA ', '#SWE ', '#LRV ', '#WO ', '#N ']:
        offset = 0
        # loop to find all instances of #N
        if typ == '#N ':
            while offset != -1:
                note, check, offset = find_note(txt, typ, offset)
                # remove spaces at the end
                note = note.rstrip()
                if check != 'pass':
                    data[0] = check
                else:
                    note = note[len(typ):]
                    nutz, check_n = find_nutz(note)
                    if check_n != 'pass':
                        data[0] = check_n
                    for n in nutz: 
                        data.append(n)
        # all instances that are not #N
        else:
            note, check, offset = find_note(txt, typ, offset)
            # remove spaces at the end
            note = note.rstrip()
            #print(check)
            if check != 'pass':
                data[0] = check
            else:
                check = check_note(note[len(typ):], typ)
                if check != 'pass':
                    data[0] = check
            #note = check_note(note[len(typ):], typ)
            data.append(note[1:])
    
    ## check if # of invalid type exists
    hashtags = [m.start() for m in re.finditer('#', txt)]
    for hashtag in hashtags:
        element = txt[hashtag:hashtag+5].split()[0]
        if (element != '#') & (element not in check_hashtype):
            data[0] = 'error: not indentified #-type - ' + element
    return data

In [20]:
cols = ['Check', 'WO', 'UZ', 'W', 'STOE', 'VTYP', 'VB', 'UENH', 'UELH',\
 'X', 'BZ', 'PZ', 'ST', 'BE', 'MA', 'SWE', 'LRV', 'XX',\
 'S_0', 'MA_0', 'Fl_0', 'LH_0', 'NH_0', 'Dring_0', 'Zeit_0', 'Bew_0', 'Sch_0', 'Rueck_0', 'Text_0',\
 'S_1', 'MA_1', 'Fl_1', 'LH_1', 'NH_1', 'Dring_1', 'Zeit_1', 'Bew_1', 'Sch_1', 'Rueck_1', 'Text_1',\
 'S_2', 'MA_2', 'Fl_2', 'LH_2', 'NH_2', 'Dring_2', 'Zeit_2', 'Bew_2', 'Sch_2', 'Rueck_2', 'Text_2', 'End'
]

In [30]:
cols = ['Check', 'WO', 'UZ', 'W', 'STOE', 'VTYP', 'VB', 'UENH', 'UELH',\
 'X', 'BZ', 'PZ', 'ST', 'BE', 'MA', 'SWE', 'LRV', 'XX',\
 'S_0', 'MA_0', 'Fl_0', 'LH_0', 'NH_0', 'Dring_0', 'Zeit_0', 'Bew_0', 'Sch_0', 'Rueck_0', 'Text_0',\
 'S_1', 'MA_1', 'Fl_1', 'LH_1', 'NH_1', 'Dring_1', 'Zeit_1', 'Bew_1', 'Sch_1', 'Rueck_1', 'Text_1', 'End'
]

In [32]:
def get_data_all(pdf_data, start=0, end=1):
    data = []
    for i in range(start, end):
        txt = pdf_data[i]['content']
        if txt[:3] == ' FB':
            data.append(get_data_page(txt))
    # convert to dataframe
    return pd.DataFrame(data, columns= cols)

In [33]:
parsed_data = get_data_all(pdf_data, 0, max_pages)
#parsed_data.to_csv(path_out)
parsed_data.to_excel(path_out)

In [19]:
parsed_data.iloc[50:100]

,0,1,2,3,4,5,6,7,8,9,...,31,32,33,34,35,36,37,38,39,40
50,pass,157 R 1,,,,,,,,,...,None,None,None,None,None,None,None,None,None,None
51,pass,157 R 2,U 200,W SSA,,,,,,,...,None,None,None,None,None,None,None,None,None,None
52,pass,157 R 3,U 200,W SSA,,,,,,,...,None,None,None,None,None,None,None,None,None,None
53,pass,157 S 0,U 200,W SSA,,,,,,,...,None,None,None,None,None,None,None,None,None,None
54,pass,157 T 0,,,,,,,,,...,None,None,None,None,None,None,None,None,None,None
55,pass,157 U 0,U 140,,,,,,,,...,None,None,None,None,None,None,None,None,None,None
56,pass,157 V 0,,,,,,,,,...,None,None,None,None,None,None,None,None,None,None
57,pass,157 W 1,U 160,,,,,,,,...,None,None,None,None,None,None,None,None,None,None
58,pass,157 W 2,,,,,,,,,...,None,None,None,None,None,None,None,None,None,None
59,pass,157 X 0,,,,,,,,,...,None,None,None,None,None,None,None,None,None,None


In [36]:
#####################################################################################

In [23]:
txt

' FB 172\n FR 02\n          Forstbetrieb Waldviertel Voralpen\n          Droß\n                                                               Druckdatum: 20.06.2021               BKL       Umtrieb          Wirtschaftswald\n                                                                                                                                                               706 B 0\n TO: 1420 Laufzeit: 01.01.2022- 31.12.2031\n                                                                Seite 101 von 1.011             9010            120            in Ertrag\n                                                                                                                                                                          2,5 ha\n    Seeh [m]           Exp           Neig [%]           Neig [°]         STOE                VTYP          Wald-Typ       Wuchs-Geb. Verbissgrad              Schälgrad        fr.SS\n      500               EB                0                0       

In [33]:
import re
hashtags = [m.start() for m in re.finditer('#', txt)]

In [34]:
for hashtag in hashtags:
    element = txt[hashtag:hashtag+4].split()[0]
    print(element)
    if (element != '#') & (element not in check_hashtype):
        print('   error')
    else:
        print('   pass')

#ST
   pass
#
   pass
#BE
   pass
#
   pass
#MA
   pass
#
   pass
#N
   pass
#
   pass
#UEN
   error
#
   pass


In [15]:
parsed_data[7] != '1'

0     False
1      True
2     False
3      True
4     False
      ...  
95    False
96    False
97     True
98     True
99     True
Name: 7, Length: 100, dtype: bool

In [24]:
mask = parsed_data[7].isin(['','1','2','3','4']) == False

In [25]:
parsed_data.loc[mask,7]

43    error:
57    error:
77        DE
Name: 7, dtype: object

In [14]:

txt = pdf_data[18]['content']

for typ in ['#BZ', '#N', '#ST', '#BE', '#MA', '#U', '#VTYP']:
    offset=0
    while offset != -1:
        xxx, offset = find_txt(txt, typ, offset)
        print(xxx)

NameError: name 'find_txt' is not defined

In [28]:
pdf_data[11]['content']

'NEU D3\n\nST große Senke, Zugweg in Mitte, stark vergast und verkrautet\nBE aus AD 2015, mit DG/KI aufgeforstet, leider Tannentrieblaus befallen an DG! einige ältere FI/TA aus Vorbestand,\neinzeln LH beigemischt\nMA AF 2015-2018, Rüsselkäferbekämpfung 2016, Schutz gegen Wild, DP Trieblausbefall ne Individuen entnehmen!\n\n8j 88 DG 10 1,0\n    5 KI 6\n    5 FI 7\n    2 TA 8\n\x0c'

In [15]:
txt.count('#BZ')

0

## Get Waldort

In [23]:
txt

' FB 171\n FR 12\n          Forstbetrieb Wienerwald\n          Pernitz\n                                                           Druckdatum: 03.05.2020               BKL       Umtrieb          Wirtschaftswald\n                                                                                                                                                            201 B 3\n TO: 1310 Laufzeit: 01.01.2021- 31.12.2030\n                                                            Seite 4 von 828                   9010          140            in Ertrag\n                                                                                                                                                                       1,98 ha\n    Seeh [m]         Exp           Neig [%]         Neig [°]         STOE                VTYP          Wald-Typ       Wuchs-Geb. Verbissgrad               Schälgrad        fr.SS\n      600            NW              58               30                 26               

In [36]:
start = txt.find('Wirtschaftswald\n')
end = txt.find('TO:')

In [42]:
wo = txt[start+17:end-2]

In [43]:
wo

'                                                                                                                                                           201 B 3'

In [44]:
wo.split()

['201', 'B', '3']

In [45]:
def get_wo(txt):
    start = txt.find('Wirtschaftswald\n')
    end = txt.find('TO:')
    wo = txt[start+17:end-2]
    return wo.split()

In [52]:
txt = pdf_data[11]['content']
get_wo(txt)

['D2',
 'da',
 'zu',
 'Wenig',
 'Platz?',
 '#BZ',
 '4FI',
 '2TA',
 '3BU',
 '#',
 '#STOE',
 '23']

In [ ]:
class OBFPdfPharser(object):

    def __init__(self, data):

        self.data = data

        # fill nan with 0
        self.data = self.data.fillna(0)


    def printit(self):
        print(self.data)
        return(self.data)


    ### 6.1.1 Einschlagsübersicht

    def fuc_tbl_hiebsatzbilanz(self, to, fr, filterx):
        '''
            self.data = input self.data (dataframe)

In [2]:
133 + 55

188

In [3]:
280 + 200

480